In [42]:
import os
import sys
sys.path.append('/kaggle/input/timm-0-6-9/pytorch-image-models-master')
import glob
import numpy as np
import pandas as pd
import random
import math
import gc
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from helper import *
tqdm.pandas()
from albumentations.pytorch import ToTensorV2
import albumentations as A
import timm
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, accuracy_score, confusion_matrix
from torch.nn import functional as F
from torch import nn
import torch
from sklearn.preprocessing import OneHotEncoder
from multiprocessing import Pool
import random
import cv2
from math import ceil
from tqdm import tqdm
import re
import matplotlib.pyplot as plt
tqdm.pandas()
from glob import glob
print('Done 1')

Done 1


In [48]:
%%time
helmet_data= pd.read_csv('./train_baseline_helmets.csv')
tracking_data = pd.read_csv('./train_player_tracking.csv')
labels = pd.read_csv('./train_labels.csv')

CPU times: user 3.65 s, sys: 632 ms, total: 4.28 s
Wall time: 4.28 s


In [49]:
%%time
label_exp = expand_contact_id(labels)
use_cols = [
    'x_position', 'y_position', 'speed', 'distance',
    'direction', 'orientation', 'acceleration', 'sa',
    'team', 'position'
]
combined_df = create_features(label_exp, tracking_data, use_cols=use_cols)

CPU times: user 27.8 s, sys: 5.16 s, total: 33 s
Wall time: 31.7 s


In [4]:
def add_bboxes_details(frame, player1, player2, game_play, view):
    '''
    takes each row in comined_df and adds the required bbox information
    (may need more processing to convert to correct datatype)
    '''
    bboxes = []
    window = 24
    query = f"frame >= {frame-window}  and frame <= {frame+window} and game_play == '{game_play}'"
    if player2 == 'G':
        query += f"  and nfl_player_id in ({player1}, 'G')"
    else:
        query += f"  and nfl_player_id in ({player1}, {player2})"
    filt = helmet_data.query(query)

    tmp = filt[filt["view"] == view]
    tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
    tmp['centre_x'] = tmp['left'] + tmp['width']/2
    tmp['centre_y'] = tmp['top'] + tmp['height']/2
    frame_list = tmp.index # these are the available frames
    for fr in range(frame - window, frame + window + 1):
        if fr in frame_list:
            x, y = tmp.loc[fr][['centre_x','centre_y']]
            bboxes.append([x, y])
        else:
            bboxes.append([np.nan, np.nan])
    
    bboxes = pd.DataFrame(np.array(bboxes)).interpolate(limit_direction='both').values
    return np.array(bboxes)
   

In [50]:
_, combined_df =train_test_split(
        combined_df, test_size=6000, random_state=69)

In [52]:
combined_df = combined_df.reset_index()

In [54]:
combined_df['bbox_endzone'] = combined_df.progress_apply(
lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Endzone'),
axis=1
    )
combined_df['bbox_sideline'] = combined_df.progress_apply(
lambda x: add_bboxes_details(x['frame'],x['nfl_player_id_1'],x['nfl_player_id_2'],x['game_play'], 'Sideline'),
axis=1
    )

100%|██████████| 6000/6000 [08:14<00:00, 12.13it/s]


In [6]:
# def read_image(path, cx, cy, view, aug):
#     img_new = np.zeros((1, 256, 256), dtype=np.float32)
#     # print(path)
#     if os.path.isfile(path):
#         if view == 'Endzone':
#             img = cv2.imread(path, 0)[
#                 cy-76:cy+180, cx-128:cx+128].copy()
#             img_new[0, :img.shape[0], :img.shape[1]] = img
#         else:
#             img = cv2.imread(path, 0)[
#                 cy-128:cy+128, cx-128:cx+128].copy()
#             img_new[0, :img.shape[0], :img.shape[1]] = img

#     return aug(image=img_new.transpose(1, 2, 0))['image'][0]

In [55]:
CFG = {
    'seed': 42,
    'num_workers':4,
    'batch_size': 64,
}

In [56]:
valid_aug = A.Compose([
    A.Normalize(mean=[0.], std=[1.]),
    ToTensorV2()
])
categorical_data_for_fitting = [
    ['home', 'CB', 'home', 'CB', ] ,
    ['home', 'DE', 'home', 'DE', ],
    ['home', 'FS', 'home', 'FS', ],
    ['home', 'TE', 'home', 'TE', ] ,
    ['home', 'ILB', 'home', 'ILB', ],
    ['home', 'OLB', 'home', 'OLB', ],
    ['home', 'T', 'home', 'T', ],
    ['home', 'G', 'home', 'G', ] ,
    ['home', 'C', 'home', 'C', ] ,
    ['home', 'QB', 'home', 'QB', ],
    ['home', 'WR', 'home', 'WR', ],
    ['home', 'RB', 'home', 'RB', ],
    ['home', 'NT', 'home', 'NT', ],
    ['home', 'DT', 'home', 'DT', ],
    ['home', 'MLB', 'home', 'MLB', ],
    ['home', 'SS', 'home', 'SS', ] ,
    ['home', 'OT', 'home', 'OT', ],
    ['home', 'LB', 'home', 'LB', ],
    ['home', 'OG', 'home', 'OG', ] ,
    ['home', 'SAF', 'home', 'SAF', ],
    ['home', 'DB', 'home', 'DB', ] ,
    ['home', 'LS', 'home', 'LS', ] ,
    ['home', 'K', 'home', 'K', ],
    ['home', 'P', 'home', 'P', ],
    ['home', 'FB', 'home', 'FB', ] ,
    ['home', 'S', 'home', 'S', ],
    ['home', 'DL', 'Ground', 'DL', ],
    ['away', 'HB', 'away', 'HB', ],
    ['away', 'HB', 'away', 'Ground', ],
]
    
one_hot = OneHotEncoder()
one_hot.fit(categorical_data_for_fitting)

OneHotEncoder()

In [57]:
class MyDataset1(Dataset):
    def __init__(self, df, aug, one_hot_transform, feature_cols=['rel_pos_x',
                                                                 'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
                                                                 'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
                                                                 'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
                                                                 'G_flug', 'orientation_1', 'orientation_2']):

        self.df = df
        self.features = feature_cols
        self.aug = aug
        self.one_hot_transform = one_hot_transform

    def __len__(self):
        return len(self.df)

    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features

    def __getitem__(self, idx):
        window = 24
        frames_to_skip = 4

        row = self.df.iloc[idx]
        mid_frame = row['frame']

        label = float(row['contact'])
        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = row['game_play'] + f'_{view}.mp4'
            frames = [mid_frame - window +
                      i for i in range(0, 2*window+1, frames_to_skip)]

            bbox_col = 'bbox_endzone' if view == 'Endzone' else 'bbox_sideline'
            bboxes = row[bbox_col][::frames_to_skip].astype(np.int32)

            if bboxes.sum() <= 0:
                imgs += [np.zeros((256, 256), dtype=np.float32)]*len(frames)
                continue

            for i, frame in enumerate(frames):
                img_new = np.zeros((256, 256), dtype=np.float32)
                cx, cy = bboxes[i]
                path = f'./work/train_frames/{video}_{frame:04d}.jpg'
                if os.path.isfile(path):
                    img_new = np.zeros((256, 256), dtype=np.float32)
                    if view == 'Endzone':
                        img = cv2.imread(path, 0)[
                            cy-76:cy+180, cx-128:cx+128].copy()
                        img_new[:img.shape[0], :img.shape[1]] = img
                    else:
                        img = cv2.imread(path, 0)[
                            cy-128:cy+128, cx-128:cx+128].copy()
                        img_new[:img.shape[0], :img.shape[1]] = img
                imgs.append(img_new)

        img = np.array(imgs).transpose(1, 2, 0)
        img = self.aug(image=img)["image"]

        features = np.array(row[self.features], dtype=np.float32)
        features[np.isnan(features)] = 0

        """
        rel_pos_x                0
        rel_pos_y                1
        rel_pos_mag              2
        rel_pos_ori              3
        rel_speed_x              4
        rel_speed_y              5
        rel_speed_mag            6
        rel_speed_ori            7
        rel_acceleration_x       8
        rel_acceleration_y       9
        rel_acceleration_mag     10
        rel_acceleration_ori     11 
        """
        if row['G_flug']:
            features[6] = row['speed_1']
            features[7] = row['direction_1']
            features[10] = row['acceleration_1']
            features[11] = row['direction_1']

            features[4] = row['speed_1']*np.sin(row['direction_1']*np.pi/180)
            features[5] = row['speed_1']*np.cos(row['direction_1']*np.pi/180)
            features[8] = row['acceleration_1'] * \
                np.sin(row['direction_1']*np.pi/180)
            features[9] = row['acceleration_1'] * \
                np.cos(row['direction_1']*np.pi/180)
        features = self.normalize_features(features)

        team_pos = np.array(
            row[['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform(
            [team_pos]
        ).toarray()[0]

        return img, torch.from_numpy(np.hstack((features, team_pos)).astype(np.float32)), torch.as_tensor(label)

In [58]:
video2helmets = {}
test_helmets_new = helmet_data.set_index('video')
for video in tqdm(helmet_data.video.unique()):
    video2helmets[video] = test_helmets_new.loc[video].reset_index(drop=True)

100%|██████████| 481/481 [00:11<00:00, 42.59it/s]


In [60]:
video2frames = {}
test_video_metadata = pd.read_csv("./train_video_metadata.csv")
for game_play in tqdm(test_video_metadata.game_play.unique()):
    for view in ['Endzone', 'Sideline']:
        video = game_play + f'_{view}.mp4'
        video2frames[video] = max(list(map(lambda x:int(x.split('_')[-1].split('.')[0]), \
                                           glob(f'./work/train_frames/{video}*'))))

100%|██████████| 240/240 [01:12<00:00,  3.32it/s]


In [64]:
class MyDataset2(Dataset):
    def __init__(self, df, aug=valid_aug, mode='test', transform=one_hot):
        self.df = df
        self.frame = df.frame.values
        feature_cols = ['rel_pos_x','rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
            'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x', 'rel_acceleration_y', 'rel_acceleration_mag',
            'rel_acceleration_ori', 'G_flug', 'orientation_1', 'orientation_2']
        self.feature = df[feature_cols].fillna(0).values
        self.players = df[['nfl_player_id_1','nfl_player_id_2']].values
        self.game_play = df.game_play.values
        self.aug = aug
        self.mode = mode
        self.one_hot_transform = transform
        
    def __len__(self):
        return len(self.df)
    
    # @lru_cache(1024)
    # def read_img(self, path):
    #     return cv2.imread(path, 0)
    def normalize_features(self, features):
        """
        normalizes the features of the players

       'rel_pos_x',
       'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
       'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
       'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori',
       'G_flug', 'orientation_1', 'orientation_2'
        """
        features /= 100
        features[3] /= 3.6
        features[7] /= 3.6
        features[11] /= 3.6
        features[13] /= 3.6
        features[14] /= 3.6
        return features
   
    def __getitem__(self, idx):   
        window = 24
        frame = self.frame[idx]

        players = []
        for p in self.players[idx]:
            if p == 'G':
                players.append(p)
            else:
                players.append(int(p))
        
        imgs = []
        for view in ['Endzone', 'Sideline']:
            video = self.game_play[idx] + f'_{view}.mp4'

            tmp = video2helmets[video]
#             tmp = tmp.query('@frame-@window<=frame<=@frame+@window')
            tmp[tmp['frame'].between(frame-window, frame+window)]
            tmp = tmp[tmp.nfl_player_id.isin(players)]#.sort_values(['nfl_player_id', 'frame'])
            tmp_frames = tmp.frame.values
            tmp = tmp.groupby('frame')[['left','width','top','height']].mean()
#0.002s

            bboxes = []
            for f in range(frame-window, frame+window+1, 1):
                if f in tmp_frames:
                    x, w, y, h = tmp.loc[f][['left','width','top','height']]
                    bboxes.append([x, w, y, h])
                else:
                    bboxes.append([np.nan, np.nan, np.nan, np.nan])
            bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
            bboxes = bboxes[::4]

            if bboxes.sum() > 0:
                flag = 1
            else:
                flag = 0
#0.03s
                    
            for i, f in enumerate(range(frame-window, frame+window+1, 4)):
                img_new = np.zeros((256, 256), dtype=np.float32)

                if flag == 1 and f <= video2frames[video]:
                    # print(f'/kaggle/work/frames/{video}_{f:04d}.jpg')
                    img = cv2.imread(f'./work/train_frames/{video}_{f:04d}.jpg', 0)

                    x, w, y, h = bboxes[i]
                    if view == 'Endzone':
                        img = img[int(y+h/2)-76:int(y+h/2)+180,int(x+w/2)-128:int(x+w/2)+128].copy()
                    else:
                        img = img[int(y+h/2)-128:int(y+h/2)+128,int(x+w/2)-128:int(x+w/2)+128].copy()
                    img_new[:img.shape[0], :img.shape[1]] = img
                    
                imgs.append(img_new)

        
        features = np.float32(self.feature[idx])
        # print(features.shape)
        # print("this is i: ",i)
        if self.df.iloc[idx]['G_flug']:
                features[ 6] = self.df.iloc[idx]['speed_1']
                features[7] = self.df.iloc[idx]['direction_1']
                features[10] = self.df.iloc[idx]['acceleration_1']
                features[11] = self.df.iloc[idx]['direction_1']

                features[4] = self.df.iloc[idx]['speed_1'] * \
                    np.sin(self.df.iloc[idx]['direction_1']*np.pi/180)
                features[5] = self.df.iloc[idx]['speed_1'] * \
                    np.cos(self.df.iloc[idx]['direction_1']*np.pi/180)
                features[8] = self.df.iloc[idx]['acceleration_1'] * \
                    np.sin(self.df.iloc[idx]['direction_1']*np.pi/180)
                features[9] = self.df.iloc[idx]['acceleration_1'] * \
                    np.cos(self.df.iloc[idx]['direction_1']*np.pi/180)
        # print("unnormalized",features)
        features = self.normalize_features(features)
        # print(features)

        team_pos = np.array(
            self.df.iloc[idx][['team_1', 'position_1', 'team_2', 'position_2']].fillna('Ground'))
        team_pos = self.one_hot_transform.transform([team_pos]).toarray()[0]

        img = np.array(imgs).transpose(1, 2, 0)    
        img = self.aug(image=img)["image"]
        label = np.float32(self.df.contact.values[idx])

        return img, torch.from_numpy(np.hstack((features, team_pos)).astype(np.float32)), label

In [65]:
test1 = MyDataset1(combined_df, aug=valid_aug, one_hot_transform=one_hot)
test2 = MyDataset2(combined_df, aug=valid_aug, transform=one_hot)

i = 13
img1, feat1, label1 = test1[i]
img2, feat2, label2 = test2[i]

print(feat1)
print(feat2)


tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0310, 0.0151, 0.0345, 0.1777, 0.0297,
        0.0145, 0.0331, 0.1777, 0.0100, 0.0636, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0310, 0.0151, 0.0345, 0.1777, 0.0297,
        0.0145, 0.0331, 0.1777, 0.0100, 0.0636, 0.0000, 1.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [66]:
class Validator1():
    def __init__(self, test_df, aug, transform):
        self.test_set = MyDataset1(
            test_df, aug=aug, one_hot_transform=transform)
        self.test_loader = DataLoader(
            self.test_set, batch_size=CFG['batch_size'], num_workers=CFG['num_workers'], shuffle=False, pin_memory=False, persistent_workers=bool(CFG['num_workers']))

    def validate(self, model):
        y_hat = []
        model.eval()
        with torch.no_grad():
            for batch in tqdm(self.test_loader):
                imgs, features, labels = batch

                imgs = imgs.to(0, non_blocking=True)
                features = features.to(0, non_blocking=True)

                preds = model(imgs, features)
                y_hat.append(preds.cpu().detach().numpy())

            y_hat = np.hstack(y_hat)

        return y_hat

In [67]:
class Validator2():
    def __init__(self, df, aug, transform, verbose=True):
        self.test_set = MyDataset2(
            df, aug=aug, transform=transform)
        self.test_loader = DataLoader(self.test_set, batch_size=CFG['batch_size'], shuffle=False,\
                                      num_workers=CFG['num_workers'], pin_memory=True)

    def validate(self, model):
        model.eval()
        y_pred = []

        with torch.no_grad():
            tk = tqdm(self.test_loader, total=len(self.test_loader))
            for step, batch in enumerate(tk):
                img, feature, label = [x.to(0) for x in batch]
                # print(feature[-1])
                output = model(img, feature).squeeze(-1)
                # print(output)
                y_pred.extend(output.cpu().numpy())

        y_pred = np.array(y_pred)
        return y_pred

In [68]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.backbone = timm.create_model(
            'resnet50', pretrained=False, num_classes=250, in_chans=26)
        self.mlp = nn.Sequential(
            nn.Linear(77, 128),
            nn.LayerNorm(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(0.2),
        )
        self.fc = nn.Linear(64+250, 1)

    def forward(self, img, feature):
        img = self.backbone(img)
        feature = self.mlp(feature)
        y = torch.sigmoid(self.fc(torch.cat([img, feature], dim=1)))
        return y.flatten()

In [69]:
model = torch.load('./model_checkpoints/resnet50_fullforce_2023-02-12-01-02-29/best_model.pth')
model.eval()

Model(
  (backbone): ResNet(
    (conv1): Conv2d(26, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act2): ReLU(inplace=True)
        (aa): Identity()
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, mome

In [70]:
valer1 = Validator1(combined_df, aug=valid_aug, transform=one_hot)

In [71]:
valer2 =  Validator2(combined_df, aug=valid_aug, transform=one_hot)

In [72]:
y_pred1 = valer1.validate(model)

100%|██████████| 94/94 [01:33<00:00,  1.01it/s]


In [73]:
y_pred2 = valer2.validate(model)

100%|██████████| 94/94 [01:54<00:00,  1.21s/it]


In [74]:
y_pred1[:3], y_pred2[:3],

(array([8.1473953e-11, 1.6620035e-04, 4.0028108e-06], dtype=float32),
 array([8.1473953e-11, 1.6620035e-04, 4.0028108e-06], dtype=float32))

In [76]:
y_fin1 = y_pred1 > 0.5
y_fin2 = y_pred2 > 0.5


In [75]:
np.sum(y_pred1 - y_pred2)

0.0

In [28]:
y_pred2

array([8.4413032e-06, 3.6112414e-08, 9.0477643e-09, ..., 3.3580517e-11,
       5.6658714e-06, 5.9381913e-05], dtype=float32)

In [77]:
labels = np.array(combined_df['contact'])

In [78]:
matthews_corrcoef(labels, y_fin1)

0.8172961250261562

In [79]:
matthews_corrcoef(labels, y_fin2)

0.8172961250261562

In [80]:
accuracy_score(labels, y_fin1), accuracy_score(labels, y_fin2)

(0.9593333333333334, 0.9593333333333334)

In [81]:
confusion_matrix(labels, y_fin1)

array([[5178,  236],
       [   8,  578]])

In [82]:
confusion_matrix(labels, y_fin2)

array([[5178,  236],
       [   8,  578]])